In [1]:
import find_eoms
import find_A_theta

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, Dataset, DataLoader

from torchvision import datasets, transforms
import matplotlib.pyplot as plt

In [51]:
import numpy as np

# Number of samples
q1_low  = -torch.pi
q1_high =  torch.pi
q2_low  = -torch.pi
q2_high =  torch.pi

n_samples = 10000

# Generate uniformly distributed points for q1 and q2
#q1 = torch.random.uniform(low=-10, high=10, size=n_samples)
#q2 = torch.random.uniform(low=-10, high=10, size=n_samples)
q1 = torch.linspace(q1_low, q1_high, n_samples)
q2 = torch.linspace(q2_low, q2_high, n_samples)

idx = torch.randperm(q2.shape[0])

q2 = q2[idx]

# Stack q1 and q2 to get the 2D coordinates
points = torch.stack([q1, q2], axis=1)  # Shape will be (1000, 2)

In [52]:
# Use TensorDataset to create the dataset
dataset = TensorDataset(points)

# Create the DataLoader with batch size and shuffling
batch_size = 64
dataloader = DataLoader(dataset, 
                        batch_size=batch_size, 
                        shuffle=True,
                        num_workers=8,
                        pin_memory=True)

print(dataloader)

# Example usage: iterate through the DataLoader
for batch in dataloader:
    #print(batch)
    break  # Just to show one batch

In [53]:
dataiter = iter(dataloader)
coordinates = next(dataiter)
print(coordinates)


[tensor([[-2.6665,  0.5231],
        [ 0.5552, -0.2573],
        [ 1.9546,  2.0514],
        [ 0.2894,  3.0115],
        [ 2.0928,  2.9028],
        [ 0.2862,  0.7393],
        [-0.5055, -2.5610],
        [ 1.9005, -0.9768],
        [-0.2617,  0.6526],
        [-1.8697, -0.2674],
        [ 2.5867,  2.0049],
        [ 1.2973, -2.0614],
        [-1.1968, -1.0748],
        [-0.8650,  1.0434],
        [ 2.0777,  1.8647],
        [-2.4994, -2.9738],
        [ 1.0126,  1.4223],
        [-0.7984,  1.3784],
        [ 2.3806, -0.1480],
        [ 0.0745,  3.0926],
        [-1.0635,  0.0600],
        [-2.8412, -1.9690],
        [ 2.2455, -0.6846],
        [-0.7739, -1.1597],
        [ 0.5860, -1.0334],
        [ 2.7734,  0.8373],
        [ 0.7531, -0.9441],
        [ 1.6354,  1.6969],
        [-2.8293, -2.6829],
        [-0.1272, -0.1574],
        [-0.8976,  2.8682],
        [ 0.7135,  1.4462],
        [ 1.0579,  0.7933],
        [-1.0717, -1.4010],
        [-2.9003, -0.9837],
        [ 2.4221,  

In [61]:
class SinCosLayer(nn.Module):
    def __init__(self):
        super(SinCosLayer, self).__init__()

    def forward(self, x):
        # Apply sin() and cos() to both coordinates
        x_sin = torch.sin(x)
        x_cos = torch.cos(x)
        x_sin_cos_shape = (x.shape[0], x.shape[1]*2)
        x_sin_cos = torch.empty(x_sin_cos_shape, dtype=x_sin.dtype, device=x.device)
        x_sin_cos[:,0::2] = x_sin
        x_sin_cos[:,1::2] = x_cos
        return x_sin_cos
    
# Custom layer to reverse the interleaved sin() and cos() back to original coordinates
class InverseSinCosLayer(nn.Module):
    def __init__(self):
        super(InverseSinCosLayer, self).__init__()

    def forward(self, x):
        # x contains interleaved sin() and cos() values
        # Assuming input is of shape (batch_size, 4) for 2D coordinates
        sin_vals = x[:, 0::2]  # Extract sin values
        cos_vals = x[:, 1::2]  # Extract cos values

        # Use atan2 to recover the original angles from sin and cos
        original_coords = torch.atan2(sin_vals, cos_vals)
        return original_coords
    



class Autoencoder(nn.Module):
    def __init__(self):
        super().__init__()
        self.encoder = nn.Sequential(
            SinCosLayer(),
            nn.Linear(4, 8),
            nn.ReLU(),
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            InverseSinCosLayer()
        )
        
        self.decoder = nn.Sequential(
            SinCosLayer(),
            nn.Linear(4, 8),
            nn.ReLU(),
            nn.Linear(8, 16),
            nn.ReLU(),
            nn.Linear(16, 8),
            nn.ReLU(),
            nn.Linear(8, 4),
            InverseSinCosLayer()
        )
    
    def forward(self, q):
        theta = self.encoder(q)
        q_hat = self.decoder(theta)
        return(theta, q_hat)

In [62]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Autoencoder().to(device)  # Move model to GPU

criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=1e-8)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.1)

In [63]:
%%time

num_epochs = 100
outputs = []
for epoch in range(num_epochs):
    for (q) in dataloader:
        q = q[0].to(device)
        latent, q_hat = model(q)
        loss = criterion(q_hat, q)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
    print(f'Epoch:{epoch+1}, Loss:{loss.item():.7f}')
    outputs.append((epoch, q, q_hat, latent))

Epoch:1, Loss:3.5251794
Epoch:2, Loss:2.0293002
Epoch:3, Loss:1.7828393
Epoch:4, Loss:3.0024273
Epoch:5, Loss:1.7157927
Epoch:6, Loss:1.7015443
Epoch:7, Loss:1.4952471
Epoch:8, Loss:2.5911641
Epoch:9, Loss:1.8954912
Epoch:10, Loss:1.7133427
Epoch:11, Loss:1.5341752
Epoch:12, Loss:1.1949530
Epoch:13, Loss:1.3747277
Epoch:14, Loss:0.7555522
Epoch:15, Loss:0.6577405
Epoch:16, Loss:0.8797487
Epoch:17, Loss:0.9797210
Epoch:18, Loss:0.9302240
Epoch:19, Loss:0.6284861
Epoch:20, Loss:1.0219303
Epoch:21, Loss:0.7859530
Epoch:22, Loss:0.5624204
Epoch:23, Loss:0.6364105
Epoch:24, Loss:1.4224169
Epoch:25, Loss:0.3021578
Epoch:26, Loss:0.5937396
Epoch:27, Loss:0.3875101
Epoch:28, Loss:0.7004519
Epoch:29, Loss:0.3569749
Epoch:30, Loss:1.2117488
Epoch:31, Loss:0.8822901
Epoch:32, Loss:0.5775625
Epoch:33, Loss:0.5264832
Epoch:34, Loss:0.7582526
Epoch:35, Loss:0.4413326
Epoch:36, Loss:0.6964914
Epoch:37, Loss:0.4207668
Epoch:38, Loss:0.4509236
Epoch:39, Loss:0.5501713
Epoch:40, Loss:0.3754048
Epoch:41,

In [64]:
print(outputs[-1][1][0:10])
print(outputs[-1][2][0:10])
print(outputs[-1][3][0:10])

tensor([[ 0.4722, -2.8067],
        [ 1.7811,  2.4334],
        [-0.1957, -1.3978],
        [ 2.8255,  0.6029],
        [-2.6213,  3.1108],
        [ 0.6815,  2.3266],
        [-2.8136,  1.5631],
        [ 1.3859, -2.5245],
        [ 2.9405, -2.3944],
        [ 1.1465, -2.6018]])
tensor([[ 0.4393, -2.1660],
        [ 1.5428,  2.1767],
        [-0.5039, -2.3007],
        [ 1.6880,  0.7513],
        [-1.7731,  2.2210],
        [ 1.5428,  2.1767],
        [-2.2257,  1.4123],
        [ 2.2427, -2.2313],
        [ 2.2594, -2.1980],
        [ 2.2844, -2.2331]], grad_fn=<SliceBackward0>)
tensor([[ 0.6793,  2.7870],
        [-1.3478, -1.7237],
        [ 1.7406,  2.5985],
        [-0.8511, -1.9222],
        [-2.8816, -1.5877],
        [-1.3527, -1.7215],
        [ 2.8084, -0.7739],
        [ 0.3438, -2.7437],
        [ 0.3065, -2.6852],
        [ 0.3345, -2.7287]], grad_fn=<SliceBackward0>)


In [50]:
for param in model.parameters():
  print(param.data)

tensor([[ 0.4520,  0.5065],
        [-0.4755,  0.2158],
        [ 0.2407,  0.2731],
        [-0.3376,  0.3557],
        [-0.3446, -0.4188],
        [-0.5234, -0.0014],
        [ 0.5522, -0.2509],
        [-0.2178,  0.5756]])
tensor([ 0.7323,  0.7177,  0.6474,  1.2978, -0.7432, -1.0371, -0.8256,  1.1215])
tensor([[ 4.2425e-01, -2.8911e-01,  1.4410e-01,  6.7605e-02, -5.3738e-01,
          8.2417e-02,  2.9009e-01, -1.9054e-01],
        [-1.2525e-03, -1.5992e-01, -4.8781e-02,  1.0470e-01, -3.8800e-07,
         -8.9580e-02, -5.1391e-36, -4.0738e-02],
        [ 1.3689e-01, -4.7365e-01,  3.6639e-02,  3.1258e-02,  8.5531e-01,
         -1.6404e-01,  1.6329e-01,  1.6068e-01],
        [ 5.6149e-02,  5.2388e-01, -2.2552e-01, -1.2172e-01,  4.5034e-01,
          7.8203e-01, -4.4133e-02,  1.3000e-01],
        [-2.3113e-01, -9.3451e-01, -1.5634e-01, -3.4218e-01,  2.4193e-01,
         -7.5661e-01, -2.1547e-01,  8.1182e-02],
        [ 3.6359e-02,  2.5997e-01, -1.1874e-01, -2.9823e-01, -7.3156e-01,
     

In [10]:
for param in model.parameters():
    print(param)

Parameter containing:
tensor([[-0.0277,  0.0314],
        [-0.0132, -0.0735],
        [ 0.6714, -0.1153],
        [-0.7665,  0.1320],
        [ 0.3084,  0.5671],
        [-0.3568, -0.6589],
        [-0.3668,  0.5146],
        [ 0.0117, -0.0446]], requires_grad=True)
Parameter containing:
tensor([-0.7584, -1.5270, -1.0428,  1.1979,  0.6490, -0.7515,  2.2004, -1.1304],
       requires_grad=True)
Parameter containing:
tensor([[-6.8646e-02, -1.8718e-02,  1.9309e-01, -9.4899e-01,  6.0808e-01,
         -3.5874e-01,  4.5148e-01, -4.0222e-01],
        [ 2.5195e-03,  3.3717e-02, -2.4525e-01,  9.9390e-01, -5.3126e-02,
         -4.8123e-02, -4.5485e-02,  7.2656e-02],
        [ 5.5440e-36, -5.4286e-36, -5.5507e-36, -5.2364e-36, -5.1896e-36,
         -5.4399e-36, -5.2684e-36, -5.5051e-36],
        [ 5.4764e-36, -5.4686e-36, -5.4230e-36, -5.4705e-36, -5.5429e-36,
         -5.4400e-36, -5.5462e-36, -5.4338e-36],
        [-5.5962e-36, -5.4270e-36, -5.2508e-36, -5.5750e-36, -5.4451e-36,
         -5.507